# INHERITANCE AND SUBCLASSES

From https://www.digitalocean.com/community/tutorials/understanding-class-inheritance-in-python-3:


"With the super() function, you can gain access to inherited methods that have been overwritten in a class object."
"The built-in Python function super() allows us to utilize parent class methods even when overriding certain aspects of those methods in our child classes." 

"When we use the super() function, we are calling a parent method into a child method to make use of it. For example, we may want to override one aspect of the parent method with certain functionality, but then call the rest of the original parent method to finish the method."

"The super() function is most commonly used within the __init__() method because that is where you will most likely need to add some uniqueness to the child class and then complete initialization from the parent."

Further reading:

https://www.digitalocean.com/community/tutorial_series/object-oriented-programming-in-python-3

## A more clear (achieved with separation of cells) version of the example that is currently in Jupyter notebook

In [ ]:
# Example taken from: https://www.python-course.eu/python3_inheritance.php

class Person:

    def __init__(self, first, last, age):
        self.firstname = first
        self.lastname = last
        self.age = age
        
    # override the str.__str__() method.
    def __str__(self):
        # when this object is called as a string (e.g., by print() or str()), return this:
        return self.firstname + " " + self.lastname + ", " + str(self.age)

    def __repr__(self):
        return self.firstname + " " + self.lastname + ", " + str(self.age)

    def constructName(self):
        return self.firstname + ' ' + self.lastname

In [11]:
Person('john', 'lokman', 32)

john lokman, 32

In [18]:
class Employee(Person):
    
    # __init__() method of Employee class overrides any other __init__() method (e.g., from Person class) that may 
    # call the Employee object.
    
    # when __init__() method is called (or when simply instantiating this class)...
    def __init__(self, first, last, age, staffnum):
        
        # initialize the superclass with the required parameters
        # and by doing so, IMPORT the superclass Person's now-initialized ATTRIBUTES and METHODS
        # after this is done, first, last, and age attributes can also be called from an Employee class instance.
        super().__init__(first, last, age)
        
        # on top of the attributes imported from the Person superclass, add this attribute
        self.staffnumber = staffnum

    # also override __str__() method of the base 'str' class and 'Person' classes (by creating a method named as '__str__()' which is 
    # the name of the methods in str and Person classed that store the string representation of the object)
    def __str__(self):
        # when this object is called as a string (e.g., by print() or str()), return this:
        return super().__str__() + ", " +  self.staffnumber

    def __repr__(self):
        # when this object is called as a string (e.g., by print() or str()), return this:
        return super().__str__() + ", " +  self.staffnumber
    
    def getEmployee(self):
        # when this method is called, self.Name will be returned for Employee class, even though .constructName is a 
        # method of the Person class—an method that is initialized and imported when previously called by 
        # Employee.__init__() method)!
        return self.constructName() + ", " +  self.staffnumber

In [23]:
Employee('john', 'doe', 36, '1000').constructName()

'john doe'

In [22]:
my_person   = Person("Marge", "Simpson", 36)
my_employee = Employee("Homer", "Simpson", 28, "1007")

my_employee.getEmployee()

'Homer Simpson, 1007'

Inherit attribute:

In [119]:
class Super_Class():
    def __init__(self, content=[]):
        self.dataset = content

class Sub_Class(Super_Class):
    def __init__(self, sub_content):
        Super_Class.__init__(self, content=sub_content)
        #super().__init__(content=sub_content)
    
    def __repr__(self):
        return repr(self.dataset)

In [120]:
Sub_Class('a')

'a'

Override method

In [172]:
class Super_Class():
    def __init__(self, content=[]):
        self.dataset = content
    
    def append(self, object):
        self.dataset.append(object)

class Sub_Class(Super_Class):
    def __init__(self, sub_content):
        Super_Class.__init__(self, content=sub_content)
        self.status = 'empty'
        
    def __repr__(self):
        return repr(self.dataset)
    
    def append(self, object):
        Super_Class.append(self, object)
        self.status = 'not empty'

In [170]:
a = Super_Class()
a.append('x')
a.dataset

['x', 'x']

In [169]:
b = Sub_Class(['a','b'])
print(b)
b.append('c', 2)
print(b)

['a', 'b']
['a', 'b', 'c']


## Subclassing str

Further reading:
- http://howto.lintel.in/python-__new__-magic-method-explained/
- http://blog.redturtle.it/2014/09/10/python-strings

The \_\_new\_\_() is called when an instance is being created. \_\_new\_\_() can be used to customize instance creation. 

When an instance is being created, \_\_new\_\_() is called before \_\_init\_\_().

### \_\_init\_\_() does not work when subclassing immutable classes

In [530]:
class BrokenLowerCaseString(str):
    """ This is going to fail!
    Source: http://blog.redturtle.it/2014/09/10/python-strings
    """
    def __init__(self, value):
        """ Return a string instance"""
        value = str(value).lower()
        str.__init__(self, value)

Will give error:

In [500]:
try:
    BrokenLowerCaseString('Alice')  #  will not convert because arguments cannot be passed to immutable class' init method
except Exception as error_message:
    print('Exception: ' + str(error_message))


Exception: object.__init__() takes no parameters


### \_\_new\_\_() must be used instead


The right way to implement this type is to override the __new__ operator instead of the __init__ one.

This is generally true for all the immutable types [1].

In [531]:
class LowerCaseString(str):
    ''' 
    Provides an object that is like a string
    but that will always be converted to lowercase
    Source: http://blog.redturtle.it/2014/09/10/python-strings
    '''
    def __new__(cls, value): # instead of __init__()
        """ Return a string instance"""
        value = str(value).lower()
        return str.__new__(cls, value) # \_\_new\_\_() should return an object

In [507]:
LowerCaseString('ABc')

'abc'

### Using \_\_new\_\_() together with \_\_init\_\_, and (content together with self)

In [519]:
class Email(str):
    """ Provides an object that is like a string but with additional attributes
    Source: http://blog.redturtle.it/2014/09/10/python-strings"""
    @staticmethod
    def _is_valid(value):
        """ Very simple validation """
        return '@' in str(value)

    def __new__(cls, value, firstname='', lastname=''):
        """ Return a string instance """
        if not cls._is_valid(value):
            raise ValueError(value)
        return str.__new__(cls, value)

    def __init__(self, value, firstname='', lastname=''):
        """ Add some attributes to the instance """
        self.firstname = str(firstname)
        self.lastname = str(lastname)

    @property
    def fullname(self):
        """ This property returns the name of the string """
        return " ".join((self.firstname, self.lastname))

In [526]:
my_email = Email('alice.burton@example.com', 'Alice', 'Burton')
my_email

'alice.burton@example.com'

In [527]:
isinstance(alice_email, str)

True

In [528]:
 alice_email.fullname

'Alice Burton'

### Using \_\_new\_\_ with \_\_init\_\_

In [1084]:
class String(str):
    ''' 
    Provides an object that is like a string
    but that will always be converted to lowercase
    Source: http://blog.redturtle.it/2014/09/10/python-strings
    '''
    def __new__(cls, content):
        """ Return a string instance"""
        content = str(content)  # for input types other than str, not a set-in-stone necessity 
        instance = str.__new__(cls, content) 
        instance.my_attribute = 'attribute one'  # this attribute will be self.my_attribute, once __init__ executes
        print('__new__ executed')
        return instance  # returns a string object with self and self.instance
    
    def __init__(self, content): # unncesessary (as my_second_attribute can also be added in __new__), but possible
        self.my_second_attribute = 'attribute two'
        self.content_parameter = content
        print('__init__ executed')

    def print_everything_inside(self):
        print('self: ', self, '    (type: ' + str(type(self)) + ')', 
              '    (isinstance(str):' + str(isinstance(self, str)) + ')', 
              '    (isinstance(String):' + str(isinstance(self, String)) + ')')
        print('self.my_attribute: ', self.my_attribute, '    (type: ' + str(type(self.my_attribute)) + ')')
        print('self.my_second_attribute: ', self.my_second_attribute, '    (type: ' + str(type(self.my_second_attribute)) + ')')
        print('self.content_parameter: ', self.content_parameter, '    (type: ' + str(type(self.content_parameter)) + ')')
        
    def print_something(something):
        print(something, type(something), '(  # "something" is the same thing with "self")')

In [1085]:
my_string = String('--1234567890--')

__new__ executed
__init__ executed


In [1086]:
my_string

'--1234567890--'

In [1087]:
print(my_string.my_attribute)
print(my_string.my_second_attribute)

attribute one
attribute two


In [1088]:
my_string.print_everything_inside()

self:  --1234567890--     (type: <class '__main__.String'>)     (isinstance(str):True)     (isinstance(String):True)
self.my_attribute:  attribute one     (type: <class 'str'>)
self.my_second_attribute:  attribute two     (type: <class 'str'>)
self.content_parameter:  --1234567890--     (type: <class 'str'>)


In [1089]:
my_string.print_something()

--1234567890-- <class '__main__.String'> (  # "something" is the same thing with "self")


In [1090]:
# basic tests
print(my_string[5] == '4' , '    indexing')
print(my_string[1:5] == '-123', '    (slicing)')
print(my_string.split('-') == ['', '', '1234567890', '', ''], '    (split)')
print(my_string.rfind('1') == 2, '    (find)')
print(my_string.lstrip('-') == '1234567890--', '    (lstrip)')
print(my_string.count('-') == 4, '    (count)')
print(my_string.replace('0', 'X') == '--123456789X--', '    (replace)')

my_string  # return test

True     indexing
True     (slicing)
True     (split)
True     (find)
True     (lstrip)
True     (count)
True     (replace)


'--1234567890--'

In [1091]:
# advanced tests
print('\nComparison with string:')
print(my_string == '--1234567890--')

print('\nhas additional attributes despite being equal to a string:')
print('my_string.my_attribute: ', my_string.my_attribute, '\n')

key = String('a')
dic = {'a':'key successfully retrieved'}
print ('\n', dic[key])


print('\n iteration:')
for each_character in my_string:
    print(each_character)


Comparison with string:
True

has additional attributes despite being equal to a string:
my_string.my_attribute:  attribute one 

__new__ executed
__init__ executed

 key successfully retrieved

 iteration:
-
-
1
2
3
4
5
6
7
8
9
0
-
-


### Simplified Version Without \_\_init\_\_ (only \_\_new\_\_)

In [1083]:
class Simple_String(str):
    ''' 
    Provides an object that is like a string
    but that will always be converted to lowercase
    Source: http://blog.redturtle.it/2014/09/10/python-strings
    '''
    def __new__(cls, content):
        """ Return a string instance"""
        content = str(content)  # this cannot be called. instead, an instance.content_parameter is created
        instance = str.__new__(cls, content) 
        instance.my_attribute = 'attribute one'  # this attribute will be self.my_attribute, once __init__ executes
                                                 # this will be the case as long as __new__ returns the 'instance')
        instance.my_second_attribute = 'attribute two'
        instance.content_parameter = content
        
        print('__new__ executed')
        return instance  # returns a string object with self and self.instance
    
    def print_everything_inside(self):
        print('self: ', self, '    (type: ' + str(type(self)) + ')', 
              '    (isinstance(str):' + str(isinstance(self, str)) + ')', 
              '    (isinstance(String):' + str(isinstance(self, String)) + ')')
        print('self.my_attribute: ', self.my_attribute, '    (type: ' + str(type(self.my_attribute)) + ')')
        print('self.my_second_attribute: ', self.my_second_attribute, '    (type: ' + str(type(self.my_second_attribute)) + ')')
        print('self.content_parameter: ', self.content_parameter, '    (type: ' + str(type(self.content_parameter)) + ')')
        
    def print_something(something):
        print(something, type(something), '(  # "something" is the same thing with "self")')

In [1072]:
my_simple_string = Simple_String('--1234567890--')

__new__ executed


In [1073]:
my_simple_string 

'--1234567890--'

In [1074]:
print(my_simple_string .my_attribute)
print(my_simple_string .my_second_attribute)

attribute one
attribute two


In [1075]:
my_simple_string.print_everything_inside()

self:  --1234567890--     (type: <class '__main__.Simple_String'>)     (isinstance(str):True)     (isinstance(String):False)
self.my_attribute:  attribute one     (type: <class 'str'>)
self.my_second_attribute:  attribute two     (type: <class 'str'>)
self.content_parameter:  --1234567890--     (type: <class 'str'>)


In [1076]:
my_simple_string.print_something()

--1234567890-- <class '__main__.Simple_String'> (  # "something" is the same thing with "self")


In [1080]:
# basic tests
print(my_simple_string[5] == '4' , '    indexing')
print(my_simple_string[1:5] == '-123', '    (slicing)')
print(my_simple_string.split('-') == ['', '', '1234567890', '', ''], '    (split)')
print(my_simple_string.rfind('1') == 2, '    (find)')
print(my_simple_string.lstrip('-') == '1234567890--', '    (lstrip)')
print(my_simple_string.count('-') == 4, '    (count)')
print(my_simple_string.replace('0', 'X') == '--123456789X--', '    (replace)')

my_simple_string  # return test

True     indexing
True     (slicing)
True     (split)
True     (find)
True     (lstrip)
True     (count)
True     (replace)


'--1234567890--'

In [1081]:
# advanced tests
print('\nComparison with string:')
print(my_simple_string == '--1234567890--')

print('\nhas additional attributes despite being equal to a string:')
print('my_string.my_attribute: ', my_simple_string.my_attribute, '\n')

key = Simple_String('a')
dic = {'a':'key successfully retrieved'}
print ('\n', dic[key])


print('\n iteration:')
for each_character in my_simple_string:
    print(each_character)


Comparison with string:
True

has additional attributes despite being equal to a string:
my_string.my_attribute:  attribute one 

__new__ executed

 key successfully retrieved

 iteration:
-
-
1
2
3
4
5
6
7
8
9
0
-
-


### Older Experiments

In [886]:
class CSV_Line(str):
    ''' 
    Provides an object that is like a string
    but that will always be converted to lowercase
    Source: http://blog.redturtle.it/2014/09/10/python-strings
    '''
    def __new__(cls, value):
        """ Return a string instance"""
        value = str(value)
        return str.__new__(cls, value)

It returns and prints as string:

In [813]:
my_csv_line = CSV_Line(' a, b, c,')
print(my_csv_line)
my_csv_line

 a, b, c,


' a, b, c,'

And it is of both str and CSV_Line class:

In [814]:
print(isinstance(my_csv_line, str))
print(isinstance(my_csv_line, CSV_Line))

True
True


String methods work on this class:

In [815]:
print(my_csv_line[1:5])
print(my_csv_line.split(','))
print(my_csv_line.rfind('b'))
print(my_csv_line.lstrip("'"))
print(my_csv_line.count(','))
print(my_csv_line.replace('a', 'X'))

a, b
[' a', ' b', ' c', '']
4
 a, b, c,
3
 X, b, c,


In [817]:
key = CSV_Line('a')
dic = {'a':1}
dic[key]

1

A more advanced version of the class, with methods and attributes (and usage of 'content' in them):

In [461]:
class CSV_Line_advanced(str):
    ''' 
    Provides an object that is like a string
    but with additional attributes and methods
    Source: http://blog.redturtle.it/2014/09/10/python-strings
    '''
    def __new__(cls, content, delimiter):
        ''' Return a string instance'''
        return str.__new__(cls, content)
    
    def __init__(self, content, delimiter):  # 'content' is needed, even though not used
        ''' Add some attributes to the instance'''
        self.delimiter = delimiter
            
    def is_string(self):  # not self!
        return isinstance(self, str)
    
    def CONVERT_to_list(self):  # not self!
        return self

In [448]:
my_csv_line_2 = CSV_Line_advanced('a, b, c, ', ',')
my_csv_line_2.CONVERT_to_list()

'a, b, c, '

In [449]:
my_csv_line_2.is_string()

True

In [450]:
print(isinstance(my_csv_line_2, str))
print(isinstance(my_csv_line_2, CSV_Line_advanced))

True
True


In [452]:
print(my_csv_line[1:5])
print(my_csv_line.split(','))
print(my_csv_line.rfind('b'))
print(my_csv_line.lstrip("'"))
print(my_csv_line.count(','))
print(my_csv_line.replace('a', 'X'))

a, b
[' a', ' b', ' c', '']
4
 a, b, c,
3
 X, b, c,


Here is a problematic way to create subclass of an immutable class:

In [260]:
class bad_CSV_Line(str):
    def __init__(self, input_string):
        str.__init__(self) # this is not needed either
        self.content = input_string # and this neither
        
    # overrides (not necessary if __new__() is used)
    def __repr__(self):
        return repr(self.content)
    def __str__(self):
        return str(self.content)
    def __getitem__(self, index):
        return self.content[index]
    def __setitem__(self, index, value):
        self.content[index] = value
    def __iter__(self):
        return iter(self.content)


It looks normal:

In [261]:
my_bad_line = bad_CSV_Line(' a, b, c ,')
my_bad_line
#'my line content'

' a, b, c ,'

In [262]:
print(my_bad_line[1:5])
print(my_bad_line.split(','))
print(my_bad_line.rfind('b'))
print(my_bad_line.lstrip("'"))
print(my_bad_line.count(','))
print(my_bad_line.replace('a', 'X'))

a, b
[' a', ' b', ' c ', '']
4
 a, b, c ,
3
 X, b, c ,


But there are certain anomalies (possibly because of some neglected overrides).